In [ ]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [206]:
from google.colab import drive
drive.mount('/content/drive')
import os
dir_path = os.chdir('/content/drive/MyDrive/Colab Notebooks/ml_plus/data')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [185]:
from surprise import BaselineOnly, Dataset, Reader, SVD, KNNBaseline
from surprise.model_selection import cross_validate
import pandas as pd
import scipy
import numpy as np

In [ ]:
ratings = pd.read_csv('my_rating.csv')

In [219]:
movies = pd.read_csv('영화 & TV 프로그램 .csv', index_col= 0)

In [220]:
movies = movies.drop(['content','imgUrl','createdYear','type'],axis=1)

In [221]:
movies = movies.drop('contentRating',axis=1)

In [222]:
movies.columns = ['movieId','title','genres']

In [ ]:
ratings = ratings.rename(columns={'contentReviewUserId':'userId'})
ratings = ratings.rename(columns={'contentId':'movieId'})
ratings = ratings.rename(columns={'userRating':'rating'})

In [191]:
movies.head(2)

,movieId,title,genres
0,631842,똑똑똑,"['Horror', 'Mystery', 'Thriller']"
1,505642,블랙 팬서: 와칸다 포에버,"['Action', 'Adventure', 'Science Fiction']"


In [192]:
ratings.head(2)

,contentReviewId,userId,movieId,title,content,rating,createdAt,updatedAt
0,1,439,93,영화93번 리뷰입니다.,재미있당.!!!,1.0,2023-03-18 15:42:42,2023-03-18 15:42:42
1,2,143,848,영화848번 리뷰입니다.,재미있당.!!!,3.9,2023-03-18 15:42:42,2023-03-18 15:42:42


In [202]:
from surprise import KNNBaseline

from surprise import SVD
from surprise import Dataset, Reader
from surprise import KNNBaseline
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import NormalPredictor
import pandas as pd
import numpy as np

# 데이터셋으로 만들기 위해 rating 정보를 저장하는 객체 생성
reader = Reader(rating_scale=(0.5, 5.0))

# 데이터셋 생성
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# SVD 알고리즘으로 모델 학습
algo_svd = SVD()
trainset, testset = train_test_split(data, test_size=.25)
algo_svd.fit(trainset)
predictions_svd = algo_svd.test(testset)

# SVD 모델의 RMSE 출력
print("SVD 모델의 RMSE:", accuracy.rmse(predictions_svd))

# KNNBaseline 알고리즘으로 모델 학습
sim_options = {'name': 'pearson_baseline', 'user_based': True}
algo_knn = KNNBaseline(sim_options=sim_options)
algo_knn.fit(trainset)
predictions_knn = algo_knn.test(testset)

# KNN 모델의 RMSE 출력
print("KNN 모델의 RMSE:", accuracy.rmse(predictions_knn))

# 전체 데이터로 모델 학습
trainset = data.build_full_trainset()
algo_svd.fit(trainset)
algo_knn.fit(trainset)

# 특정 유저의 영화 평점 정보를 가져옴
user_id = 5
user_ratings = ratings[ratings.userId == user_id][['movieId', 'rating']]

# SVD 모델과 KNN 모델의 예측값 계산
svd_estimates = []
knn_estimates = []
for movie_id in ratings.movieId.unique():
    # SVD 모델의 예측값 계산
    svd_estimate = algo_svd.predict(user_id, movie_id).est
    svd_estimates.append(svd_estimate)

    # KNN 모델의 예측값 계산
    knn_estimate = algo_knn.predict(user_id, movie_id).est
    knn_estimates.append(knn_estimate)

# 예측값을 이용해 hybrid 점수 계산
alpha = 0.5
hybrid_scores = alpha * np.array(svd_estimates) + (1 - alpha) * np.array(knn_estimates)

# hybrid 점수를 이용해 영화 추천
recommended_movie_ids = []
for movie_id, hybrid_score in sorted(zip(ratings.movieId.unique(), hybrid_scores), key=lambda x: x[1], reverse=True)[:10]:
    if movie_id not in user_ratings.movieId.values:
        recommended_movie_ids.append(movie_id)

# 추천 영화 출력
print("hybrid 모델의 추천 영화:", recommended_movie_ids)

RMSE: 1.1912
SVD 모델의 RMSE: 1.1912027079553327
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.2214
KNN 모델의 RMSE: 1.2214076056791625
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
hybrid 모델의 추천 영화: [756, 822, 132, 195, 416, 147]


In [208]:
import pickle

# 모델 저장
file_path = os.path.join(os.getcwd(), 'hybrid_model.pkl')
with open(file_path, 'wb') as f:
    pickle.dump((algo_svd, algo_knn, alpha), f)

In [212]:
# 모델 불러오기
with open('hybrid_model.pkl', 'rb') as f:
    algo_svd, algo_knn, alpha = pickle.load(f)

In [227]:
# 불러오기 후 사용방법

# 특정 유저의 아이디와 기존 평가한 영화 리스트
user_id = 5
rated_movies = [570, 648, 849, 1197, 1265, 1387, 1750, 2019, 2571, 2692]

# 기존에 평가한 영화를 제외한 추천 영화 리스트 생성
recommended_movies = {}
for movie_id in ratings['movieId'].unique():
    if movie_id not in rated_movies:
        # SVD 모델과 KNN 모델의 예측값 계산
        svd_estimate = algo_svd.predict(user_id, movie_id).est
        knn_estimate = algo_knn.predict(user_id, movie_id).est
        
        # 예측값을 이용해 hybrid 점수 계산
        hybrid_score = alpha * svd_estimate + (1 - alpha) * knn_estimate
        
        recommended_movies[movie_id] = hybrid_score

# 추천 영화 리스트를 점수(score)를 기준으로 내림차순 정렬
recommended_movies = sorted(recommended_movies.items(), key=lambda x: x[1], reverse=True)

# 추천 영화 리스트에서 상위 10개의 영화 추출
top_movies = recommended_movies[:10]

# 추천 영화 출력
print(f"사용자 {user_id}에게 추천하는 영화 리스트:")
for i, movie in enumerate(top_movies):
    print(f"{i+1}. 영화 ID: {movie[0]}, 예측 평점: {movie[1]}")

사용자 5에게 추천하는 영화 리스트:
1. 영화 ID: 756, 예측 평점: 4.207481926138557
2. 영화 ID: 822, 예측 평점: 4.194241894928606
3. 영화 ID: 132, 예측 평점: 4.191445297326521
4. 영화 ID: 195, 예측 평점: 4.130236158462914
5. 영화 ID: 416, 예측 평점: 4.113644866206802
6. 영화 ID: 8, 예측 평점: 4.110514702175426
7. 영화 ID: 147, 예측 평점: 4.105772216135797
8. 영화 ID: 136, 예측 평점: 4.08771854922215
9. 영화 ID: 115, 예측 평점: 4.073361820319465
10. 영화 ID: 969, 예측 평점: 4.0602680833499125
